In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/My Drive/Research Project

/content/drive/My Drive/Research Project


In [ ]:
import pandas as pd
import numpy as np
import os
# import src
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
pip install -U mittens

Requirement already up-to-date: mittens in /usr/local/lib/python3.6/dist-packages (0.2)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from statistics import mean, stdev, median, mode
# With PCA
from sklearn.decomposition import PCA

from sklearn.feature_extraction.text import CountVectorizer
# SVM
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
# KNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from mittens import GloVe, Mittens

In [ ]:
# pip install autocorrect

In [ ]:

import nltk
import string
from nltk.tokenize import word_tokenize
import random
import pickle
from nltk.corpus import stopwords

# from autocorrect import Speller
# from pycontractions import Contractions

# from spellchecker import SpellChecker

import re
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer 

from hyperopt import fmin, tpe, hp

In [ ]:
nltk.download('punk')

[nltk_data] Error loading punk: Package 'punk' not found in index


False

In [ ]:
from numpy import asarray
from numpy import zeros
from sklearn.utils import shuffle
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

In [ ]:
# nltk.download('all')


## Loading Data

In [ ]:
data_path = r'/content/drive/My Drive/Research Project/Data'

In [ ]:
# data_path = r'C:\Users\kalya\OneDrive - University of Illinois at Chicago\!UIC\!Semesters\3rd Sem\CS 583 Data Mining and Text Mining\Research Project\Data'

In [ ]:
train1_pr_df = pd.read_csv(os.path.join(data_path, 'Obama Training1 Data.csv'), usecols = [1,2])
# romney_train_pr_df = pd.read_csv(os.path.join(data_path, 'Romney Training1 Data.csv'), usecols = [1,2])

In [ ]:
train1_pr_df.head()

,Doc Text,Sentiment
0,kirkpatrick wear baseball cap embroider obama ...,Neutral
1,obama debate cracker as cracker tonight tune t...,Positive
2,miss point afraid understand big picture dont ...,Neutral
3,raise democrat leave party year ago lifetime n...,Negative
4,obama camp can not afford low expectation toni...,Neutral


In [ ]:
train1_pr_df = train1_pr_df.dropna()
# romney_train_pr_df = romney_train_pr_df.dropna()

### Converting into TF-IDF

In [ ]:
def tf_idf(x):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(x)
    #print(tfidf_matrix)
    x1 = tfidf_matrix.toarray()
#     print(x1)
    return x1

In [ ]:
# train1_pr_tfidf = tf_idf(train1_pr_df['Doc Text'])
# # romney_train_pr_tfidf = tf_idf(romney_train_pr_df['Doc Text'])

In [ ]:
# train1_pr_tfidf.shape

### Cross Valid Function

In [ ]:
def cross_valid(X,y,model):
    f1_Positive  =[]
    f1_Neutral =[]
    f1_Negative =[]
    acc =[]
    cv = KFold(n_splits=5,shuffle=True)
    for train_index, val_index in cv.split(X):
    #     print("Train Index: ", train_index, "\n")
    #     print("Test Index: ", test_index)
    #     print(X.iloc[train_index])
    #     print(f)
        # print(1)
        X_train1, X_val, y_train1, y_val = X[train_index], X[val_index], y.iloc[train_index], y.iloc[val_index]
        temp = y_train1.apply(lambda x: 1 if x == 'Positive' else (3 if x == 'Negative' else 2))

        vectorizer = TfidfVectorizer()
        tfidf_matrix = vectorizer.fit_transform(X_train1)
        #print(tfidf_matrix)
        X_train1_tfidf = tfidf_matrix.toarray()

        model = model.fit(X_train1_tfidf, temp)

        tfidf_matrix2 = vectorizer.transform(X_val)
        #print(tfidf_matrix)
        X_val_tfidf = tfidf_matrix2.toarray()

        y_pred = model.predict(X_val_tfidf)
        temp_y_val = y_val.apply(lambda x: 1 if x == 'Positive' else (3 if x == 'Negative' else 2))
        f1_temp = f1_score(temp_y_val, y_pred, average = None)

        f1_Positive.append(f1_temp[0])
        f1_Neutral.append(f1_temp[1])
        f1_Negative.append(f1_temp[2])

        acc.append(round(accuracy_score(temp_y_val, y_pred),3))

    return round(mean(acc),3), round(mean(f1_Positive),3), round(mean(f1_Neutral),3), round(mean(f1_Negative),3)

### Cross Valid func with PCA

In [ ]:
def cross_valid_PCA(X,y,model, n):
    f1_Positive  =[]
    f1_Neutral =[]
    f1_Negative =[]
    acc =[]
    cv = KFold(n_splits=5,shuffle=True)
    
    pca = PCA(n_components=n)
    
    
    for train_index, val_index in cv.split(X):
    #     print("Train Index: ", train_index, "\n")
    #     print("Test Index: ", test_index)
    #     print(X.iloc[train_index])
    #     print(f)
        # print(1)
        X_train1, X_val, y_train1, y_val = X[train_index], X[val_index], y.iloc[train_index], y.iloc[val_index]

        vectorizer = TfidfVectorizer()
        tfidf_matrix = vectorizer.fit_transform(X_train1)
        #print(tfidf_matrix)
        X_train1_tfidf = tfidf_matrix.toarray()

        x_pca1 = pca.fit_transform(X_train1_tfidf)
        temp = y_train1.apply(lambda x: 1 if x == 'Positive' else (3 if x == 'Negative' else 2))

        model = model.fit(x_pca1, temp)

        tfidf_matrix2 = vectorizer.transform(X_val)
        #print(tfidf_matrix)
        X_val_tfidf = tfidf_matrix2.toarray()
        
        x_pca_val = pca.transform(X_val_tfidf)
        y_pred = model.predict(x_pca_val)
        temp_y_val = y_val.apply(lambda x: 1 if x == 'Positive' else (3 if x == 'Negative' else 2))
        f1_temp = f1_score(temp_y_val, y_pred, average = None)

        f1_Positive.append(f1_temp[0])
        f1_Neutral.append(f1_temp[1])
        f1_Negative.append(f1_temp[2])

        acc.append(round(accuracy_score(temp_y_val, y_pred),3))

    return round(mean(acc),3), round(mean(f1_Positive),3), round(mean(f1_Neutral),3), round(mean(f1_Negative),3)

### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:


# # clf = SVC(kernel="rbf", gamma=1, class_weight='balanced')
# #     error = 1-(cross_val_score(clf, x_pca1, temp, cv = 5, scoring = 'f1_macro'))
# #     f1 = mean(error) + stdev(error) 
# #print(f1)


# clf = LogisticRegression(class_weight='balanced')
# accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid(obama_train_pr_tfidf,obama_train_pr_df['Sentiment'], clf) 
# eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)
# eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}
# # print([f1_list[2],f1_list[1],f1_list[0]])

# print(eval_dict)


In [ ]:
# With Bayesian Optimization, and LR

def objective_func_LR(args):

    C = args['C']
    # penalty = args['penalty']
    # multi_class = args['multi_class']
    # l1_ratio = args['l1_ratio']

    print(C)
    clf = LogisticRegression(C= C,class_weight='balanced',n_jobs =-1)
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid(train1_pr_df['Doc Text'],train1_pr_df['Sentiment'], clf) 
    eval_score = round(mean([0.4*accuracy, 1.6*f1_Positive]),3)
    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}
    print(eval_dict)
    return -(eval_score)


space = {'C': hp.uniform('C', 0,2),
        # 'penalty': hp.choice('penalty', ['l1', 'l2']), 
        # 'multi_class': hp.choice('multi_class',['ovr', 'multinomial']),

        #  'l1_ratio' : hp.uniform('l1_ratio',0,1)
        }                        
                                
                                
best_classifier_LR = fmin(objective_func_LR, space, algo=tpe.suggest, max_evals=10)
print(best_classifier_LR)

0.23478032266387028
{'accuracy': 0.589, 'f1_pos': 0.603, 'f1_neu': 0.541, 'f1_neg': 0.62, 'eval_score': 0.6}
1.4213004007429328
{'accuracy': 0.592, 'f1_pos': 0.607, 'f1_neu': 0.544, 'f1_neg': 0.625, 'eval_score': 0.604}
0.2824079082273807
{'accuracy': 0.585, 'f1_pos': 0.605, 'f1_neu': 0.529, 'f1_neg': 0.62, 'eval_score': 0.601}
0.29672536053974863
{'accuracy': 0.584, 'f1_pos': 0.601, 'f1_neu': 0.531, 'f1_neg': 0.618, 'eval_score': 0.598}
0.6297337569771542
{'accuracy': 0.595, 'f1_pos': 0.608, 'f1_neu': 0.547, 'f1_neg': 0.625, 'eval_score': 0.605}
1.2647248746778286
{'accuracy': 0.584, 'f1_pos': 0.602, 'f1_neu': 0.527, 'f1_neg': 0.621, 'eval_score': 0.598}
1.0189797478804836
{'accuracy': 0.587, 'f1_pos': 0.609, 'f1_neu': 0.537, 'f1_neg': 0.614, 'eval_score': 0.605}
0.08511065203177037
{'accuracy': 0.58, 'f1_pos': 0.593, 'f1_neu': 0.525, 'f1_neg': 0.618, 'eval_score': 0.59}
0.1991214452005865
{'accuracy': 0.585, 'f1_pos': 0.602, 'f1_neu': 0.529, 'f1_neg': 0.62, 'eval_score': 0.599}
0.565

In [ ]:
0.6297337569771542
{'accuracy': 0.595, 'f1_pos': 0.608, 'f1_neu': 0.547, 'f1_neg': 0.625, 'eval_score': 0.605}

{'accuracy': 0.59,
 'eval_score': 0.608,
 'f1_neg': 0.621,
 'f1_neu': 0.535,
 'f1_pos': 0.613}

In [ ]:
# With PCA, Bayesian Optimization, and LR

def objective_func_LR_PCA(args):

    C = args['C']
    n = args['n']
    
    # penalty = args['penalty']
    # multi_class = args['multi_class']
    # l1_ratio = args['l1_ratio']


    clf = LogisticRegression(C= C,class_weight='balanced',n_jobs =-1)
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid_PCA(train1_pr_df['Doc Text'],train1_pr_df['Sentiment'], clf, n) 
    eval_score = round(mean([0.4*accuracy, 1.6*f1_Positive]),3)
    par_dict = {'C': C, 'n': n}
    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}
    print(par_dict)
    print(eval_dict)
    print('\n')    
    return -(eval_score)


space = {'C': hp.uniform('C', 0,2),
         'n': hp.choice('n', np.arange(100, 500, step =50))
        # 'penalty': hp.choice('penalty', ['l1', 'l2']), 
        # 'multi_class': hp.choice('multi_class',['ovr', 'multinomial']),
        #  'l1_ratio' : hp.uniform('l1_ratio',0,1)
        }                        
                                
                                
best_classifier_LR_PCA = fmin(objective_func_LR_PCA, space, algo=tpe.suggest, max_evals=20)


{'C': 1.6242297014560634, 'n': 200}
{'accuracy': 0.562, 'f1_pos': 0.577, 'f1_neu': 0.516, 'f1_neg': 0.591, 'eval_score': 0.574}
{'C': 0.7309407153863505, 'n': 200}
{'accuracy': 0.562, 'f1_pos': 0.574, 'f1_neu': 0.516, 'f1_neg': 0.594, 'eval_score': 0.572}
{'C': 0.3418576542156955, 'n': 250}
{'accuracy': 0.561, 'f1_pos': 0.578, 'f1_neu': 0.513, 'f1_neg': 0.592, 'eval_score': 0.575}
{'C': 0.6982693845065415, 'n': 450}
{'accuracy': 0.576, 'f1_pos': 0.594, 'f1_neu': 0.526, 'f1_neg': 0.608, 'eval_score': 0.59}
{'C': 0.006619501541552442, 'n': 450}
{'accuracy': 0.562, 'f1_pos': 0.57, 'f1_neu': 0.491, 'f1_neg': 0.61, 'eval_score': 0.568}
{'C': 0.11366735027234864, 'n': 400}
{'accuracy': 0.571, 'f1_pos': 0.584, 'f1_neu': 0.521, 'f1_neg': 0.604, 'eval_score': 0.581}
{'C': 0.42724669077849775, 'n': 350}
{'accuracy': 0.568, 'f1_pos': 0.585, 'f1_neu': 0.518, 'f1_neg': 0.599, 'eval_score': 0.582}
{'C': 1.179152671926634, 'n': 400}
{'accuracy': 0.574, 'f1_pos': 0.599, 'f1_neu': 0.523, 'f1_neg': 0.60

In [ ]:
{'C': 1.179152671926634, 'n': 400}
{'accuracy': 0.574, 'f1_pos': 0.599, 'f1_neu': 0.523, 'f1_neg': 0.601, 'eval_score': 0.594}

{'accuracy': 0.58,
 'eval_score': 0.592,
 'f1_neg': 0.612,
 'f1_neu': 0.53,
 'f1_pos': 0.595}

In [ ]:
# # With Bayesian Optimization, different class weights and LR

# def objective_func_LR(args):

#     C = args['C']
#     balance = args['balance']
#     # multi_class = args['multi_class']
#     # l1_ratio = args['l1_ratio']

 
#     clf = LogisticRegression(C= C,class_weight=balance,n_jobs =-1)
#     accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid(obama_train_pr_tfidf,obama_train_pr_df['Sentiment'], clf) 
#     eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)
#     eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}
#     print(eval_dict)
#     return -(eval_score)


# space = {'C': hp.uniform('C', 0,10),
#          'balance': hp.choice('balance', [{1:3, 2:1, 3:2}, {1:15, 2:1, 3:10}, {1:20, 2:5, 3:10}, {1:100, 2:10, 3:100}])
#         # 'penalty': hp.choice('penalty', ['l1', 'l2']), 
#         # 'multi_class': hp.choice('multi_class',['ovr', 'multinomial']),

#         #  'l1_ratio' : hp.uniform('l1_ratio',0,1)
#         }                        
                                
                                
# best_classifier_LR = fmin(objective_func_LR, space, algo=tpe.suggest, max_evals=20)
# print(best_classifier_LR)

### SVM

In [ ]:
# clf = SVC(kernel = 'rbf', class_weight='balanced')
# accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid(train1_pr_df['Doc Text'],train1_pr_df['Sentiment'], clf) 
# eval_score = round(mean([0.4*accuracy, 1.6*f1_Positive]),3)

# eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}
# print(eval_dict)

In [ ]:
# # With Bayesian Optimization, and SVM

# def objective_func_SVM(args):
#     C = args['C']
# #     kernel = args['kernel']
# #     gamma = args['gamma']
# #     degree = args['degree']

#     par_dict = {'C': C}
#     print(par_dict)
 
#     clf = SVC(C= C, kernel = 'rbf', class_weight='balanced')
#     accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid(train1_pr_tfidf,train1_pr_df['Sentiment'], clf) 
#     eval_score = round(mean([0.4*accuracy, 1.6*f1_Positive]),3)

#     eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}
#     print(eval_dict)
#     print('\n')    
#     return -(eval_score)


# space = {'C': hp.uniform('C', 0,20)
# #         'kernel': hp.choice('kernel', ['poly', 'rbf']) 
# #         'gamma': hp.choice('gamma',range(1,4)),
# #          'degree' : hp.choice('degree',range(1,4))
#         }
                                
                                
                                
# best_classifier_SVM_PCA = fmin(objective_func_SVM, space, algo=tpe.suggest, max_evals=20)
# print(best_classifier_SVM_PCA)

In [ ]:
# With PCA, Bayesian Optimization, and SVM

def objective_func_SVM_PCA(args):
    C = args['C']
    n = args['n']
#     kernel = args['kernel']
#     gamma = args['gamma']
#     degree = args['degree']

    par_dict = {'C': C, 'n': n, 'kernel': 'rbf'}
    print(par_dict)
 
    clf = SVC(C= C, kernel = 'rbf', class_weight='balanced')
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid_PCA(train1_pr_df['Doc Text'],train1_pr_df['Sentiment'], clf, n) 
    eval_score = round(mean([0.4*accuracy, 1.6*f1_Positive]),3)

    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}
    print(eval_dict)
    print('\n')    
    return -(eval_score)


space = {'C': hp.uniform('C', 0,1),
         'n': hp.choice('n', np.arange(3, 1003, step =100))
#         'kernel': hp.choice('kernel', ['poly', 'rbf']) 
#         'gamma': hp.choice('gamma',range(1,4)),
#          'degree' : hp.choice('degree',range(1,4))
        }
                                
                                
                                
best_classifier_SVM_PCA = fmin(objective_func_SVM_PCA, space, algo=tpe.suggest, max_evals=20)
print(best_classifier_SVM_PCA)

{'C': 0.6972208032409349, 'n': 303, 'kernel': 'rbf'}
{'accuracy': 0.577, 'f1_pos': 0.596, 'f1_neu': 0.541, 'f1_neg': 0.595, 'eval_score': 0.592}
{'C': 0.225667574634682, 'n': 803, 'kernel': 'rbf'}
{'accuracy': 0.567, 'f1_pos': 0.556, 'f1_neu': 0.534, 'f1_neg': 0.605, 'eval_score': 0.558}
{'C': 0.46701118884195203, 'n': 703, 'kernel': 'rbf'}
{'accuracy': 0.581, 'f1_pos': 0.592, 'f1_neu': 0.549, 'f1_neg': 0.606, 'eval_score': 0.59}
{'C': 0.6676113411219312, 'n': 803, 'kernel': 'rbf'}
{'accuracy': 0.587, 'f1_pos': 0.594, 'f1_neu': 0.549, 'f1_neg': 0.619, 'eval_score': 0.593}
{'C': 0.3915373740566396, 'n': 603, 'kernel': 'rbf'}
{'accuracy': 0.584, 'f1_pos': 0.592, 'f1_neu': 0.542, 'f1_neg': 0.618, 'eval_score': 0.59}
{'C': 0.40950567689882433, 'n': 103, 'kernel': 'rbf'}
{'accuracy': 0.551, 'f1_pos': 0.559, 'f1_neu': 0.511, 'f1_neg': 0.58, 'eval_score': 0.557}
{'C': 0.341220099699022, 'n': 3, 'kernel': 'rbf'}
{'accuracy': 0.476, 'f1_pos': 0.512, 'f1_neu': 0.381, 'f1_neg': 0.525, 'eval_score

In [ ]:
{'C': 0.844193832928247, 'n': 603, 'kernel': 'rbf'}
{'accuracy': 0.584, 'f1_pos': 0.607, 'f1_neu': 0.542, 'f1_neg': 0.605, 'eval_score': 0.602}

{'accuracy': 0.59,
 'eval_score': 0.599,
 'f1_neg': 0.605,
 'f1_neu': 0.568,
 'f1_pos': 0.601}

### KNN

In [ ]:
# WIth Bayesian Optimization and KNN
def objective_func_KNN(args):
    n_neighbors = args['n_neighbors']
    metric = args['metric']  

    par_dict = {'n_neighbors': n_neighbors, 'metric': metric}
    print(par_dict)
    
    clf = KNeighborsClassifier(n_neighbors=n_neighbors, metric=metric, n_jobs = -1)
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid(train1_pr_df['Doc Text'],train1_pr_df['Sentiment'], clf) 
    eval_score = round(mean([0.4*accuracy, 1.6*f1_Positive]),3)

    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}

    print(eval_dict)
    print('\n')    
    return -(eval_score)

space = {'n_neighbors': hp.choice('n_neighbors',np.arange(1,31, step =3)),
        'metric':hp.choice('metric', ["euclidean","manhattan"])
        }

best_classifier_KNN = fmin(objective_func_KNN, space, algo=tpe.suggest, max_evals=10)
print(best_classifier_KNN)

{'n_neighbors': 1, 'metric': 'manhattan'}
{'accuracy': 0.461, 'f1_pos': 0.508, 'f1_neu': 0.431, 'f1_neg': 0.425, 'eval_score': 0.499}
{'n_neighbors': 19, 'metric': 'manhattan'}
{'accuracy': 0.367, 'f1_pos': 0.485, 'f1_neu': 0.34, 'f1_neg': 0.055, 'eval_score': 0.461}
{'n_neighbors': 4, 'metric': 'euclidean'}
{'accuracy': 0.492, 'f1_pos': 0.557, 'f1_neu': 0.431, 'f1_neg': 0.462, 'eval_score': 0.544}
{'n_neighbors': 4, 'metric': 'manhattan'}
{'accuracy': 0.427, 'f1_pos': 0.516, 'f1_neu': 0.365, 'f1_neg': 0.31, 'eval_score': 0.498}
{'n_neighbors': 16, 'metric': 'manhattan'}
{'accuracy': 0.364, 'f1_pos': 0.486, 'f1_neu': 0.299, 'f1_neg': 0.075, 'eval_score': 0.462}
{'n_neighbors': 25, 'metric': 'euclidean'}
{'accuracy': 0.514, 'f1_pos': 0.567, 'f1_neu': 0.409, 'f1_neg': 0.535, 'eval_score': 0.556}
{'n_neighbors': 10, 'metric': 'manhattan'}
{'accuracy': 0.383, 'f1_pos': 0.495, 'f1_neu': 0.307, 'f1_neg': 0.177, 'eval_score': 0.473}
{'n_neighbors': 7, 'metric': 'euclidean'}
{'accuracy': 0.51,

In [ ]:
# WIth PCA , Bayesian Optimization and KNN
def objective_func_KNN_PCA(args):
    n_neighbors = args['n_neighbors']
    metric = args['metric']
    n = args['n']    

    par_dict = {'n_neighbors': n_neighbors, 'n': n, 'metric': metric}
    print(par_dict)
    
    clf = KNeighborsClassifier(n_neighbors=n_neighbors, metric=metric, n_jobs = -1)
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid_PCA(train1_pr_df['Doc Text'],train1_pr_df['Sentiment'], clf, n) 
    eval_score = round(mean([0.4*accuracy, 1.6*f1_Positive]),3)

    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}

    print(eval_dict)
    print('\n')    
    return -(eval_score)

space = {'n_neighbors': hp.choice('n_neighbors',np.arange(1,10, step =1)),
        'metric':hp.choice('metric', ["euclidean","manhattan"]),
        'n': hp.choice('n', np.arange(3,63, step =5))}

best_classifier_KNN_PCA = fmin(objective_func_KNN_PCA, space, algo=tpe.suggest, max_evals=20)


{'n_neighbors': 1, 'n': 58, 'metric': 'manhattan'}
{'accuracy': 0.463, 'f1_pos': 0.463, 'f1_neu': 0.456, 'f1_neg': 0.47, 'eval_score': 0.463}
{'n_neighbors': 2, 'n': 38, 'metric': 'manhattan'}
{'accuracy': 0.439, 'f1_pos': 0.501, 'f1_neu': 0.443, 'f1_neg': 0.334, 'eval_score': 0.489}
{'n_neighbors': 4, 'n': 33, 'metric': 'euclidean'}
{'accuracy': 0.466, 'f1_pos': 0.492, 'f1_neu': 0.468, 'f1_neg': 0.437, 'eval_score': 0.487}
{'n_neighbors': 3, 'n': 8, 'metric': 'euclidean'}
{'accuracy': 0.44, 'f1_pos': 0.487, 'f1_neu': 0.398, 'f1_neg': 0.43, 'eval_score': 0.478}
{'n_neighbors': 6, 'n': 8, 'metric': 'manhattan'}
{'accuracy': 0.454, 'f1_pos': 0.49, 'f1_neu': 0.434, 'f1_neg': 0.439, 'eval_score': 0.483}
{'n_neighbors': 7, 'n': 58, 'metric': 'manhattan'}
{'accuracy': 0.473, 'f1_pos': 0.483, 'f1_neu': 0.487, 'f1_neg': 0.447, 'eval_score': 0.481}
{'n_neighbors': 9, 'n': 33, 'metric': 'manhattan'}
{'accuracy': 0.486, 'f1_pos': 0.5, 'f1_neu': 0.487, 'f1_neg': 0.471, 'eval_score': 0.497}
{'n_nei

### Naive Bayes

In [ ]:
clf = GaussianNB()
accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid(train1_pr_df['Doc Text'],train1_pr_df['Sentiment'], clf) 
eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)
eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}
print(eval_dict)

{'accuracy': 0.476, 'f1_pos': 0.536, 'f1_neu': 0.358, 'f1_neg': 0.5, 'eval_score': 0.504}


In [ ]:
# With PCA and Naive bayes
def objective_func_NB_PCA(args):
    n = args['n']

    par_dict = {'n': n}
    print(par_dict)
    
    clf = GaussianNB()
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid_PCA(train1_pr_df['Doc Text'],train1_pr_df['Sentiment'], clf, n) 
    eval_score = round(mean([0.4*accuracy, 1.6*f1_Positive]),3)
    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}

    print(eval_dict)
    print('\n')    
    return -(eval_score)

space = {'n': hp.choice('n', np.arange(3,63, step =5))}

best_classifier_NB_PCA = fmin(objective_func_NB_PCA, space, algo=tpe.suggest, max_evals=10)



{'n': 53}
{'accuracy': 0.485, 'f1_pos': 0.46, 'f1_neu': 0.428, 'f1_neg': 0.546, 'eval_score': 0.465}
{'n': 13}
{'accuracy': 0.458, 'f1_pos': 0.449, 'f1_neu': 0.311, 'f1_neg': 0.546, 'eval_score': 0.451}
{'n': 18}
{'accuracy': 0.47, 'f1_pos': 0.459, 'f1_neu': 0.34, 'f1_neg': 0.554, 'eval_score': 0.461}
{'n': 58}
{'accuracy': 0.484, 'f1_pos': 0.469, 'f1_neu': 0.447, 'f1_neg': 0.527, 'eval_score': 0.472}
{'n': 23}
{'accuracy': 0.471, 'f1_pos': 0.46, 'f1_neu': 0.36, 'f1_neg': 0.55, 'eval_score': 0.462}
{'n': 53}
{'accuracy': 0.474, 'f1_pos': 0.456, 'f1_neu': 0.426, 'f1_neg': 0.526, 'eval_score': 0.46}
{'n': 13}
{'accuracy': 0.459, 'f1_pos': 0.45, 'f1_neu': 0.312, 'f1_neg': 0.547, 'eval_score': 0.452}
{'n': 33}
{'accuracy': 0.472, 'f1_pos': 0.445, 'f1_neu': 0.407, 'f1_neg': 0.538, 'eval_score': 0.45}
{'n': 38}
{'accuracy': 0.472, 'f1_pos': 0.455, 'f1_neu': 0.404, 'f1_neg': 0.538, 'eval_score': 0.458}
{'n': 53}
{'accuracy': 0.483, 'f1_pos': 0.458, 'f1_neu': 0.428, 'f1_neg': 0.542, 'eval_scor

## Ensemble Methods

### Random Forest

In [ ]:
# With Bayesian Optimization, and Random Forest

def objective_func_RF(args):
    n_estimators = args['n_estimators']
    max_depth = args['max_depth']
    min_samples_split = args['min_samples_split']
    min_samples_leaf = args['min_samples_leaf']

    par_dict = {'n_estimators': n_estimators, 'max_depth': max_depth, 'min_samples_split': min_samples_split, 'min_samples_leaf': min_samples_leaf}
    print(par_dict)
    
    clf = RandomForestClassifier(n_estimators = n_estimators, max_depth = max_depth, min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split, n_jobs = -1)
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid(train1_pr_df['Doc Text'],train1_pr_df['Sentiment'], clf) 
    eval_score = round(mean([0.4*accuracy, 1.6*f1_Positive]),3)
    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}

    print(eval_dict)
    print('\n')    
    return -(eval_score)

space = {'n_estimators': hp.choice('n_estimators', np.arange(20,101, step =1)), 
         'max_depth': hp.choice('max_depth', np.arange(20,101, step =1)), 
#         'criterion': hp.choice('criterion',['gini', 'entropy']),
#         'max_features': hp.choice('max_features', ['auto', 'sqrt']),
        'min_samples_split': hp.choice('min_samples_split', np.arange(2,12, step =1)),
         'min_samples_leaf' : hp.choice('min_samples_leaf',np.arange(1,12, step =1))
        }
                                
                                
                                
best_classifier_RF = fmin(objective_func_RF, space, algo=tpe.suggest, max_evals=25)
print(best_classifier_RF)

{'n_estimators': 67, 'max_depth': 39, 'min_samples_split': 4, 'min_samples_leaf': 6}
{'accuracy': 0.562, 'f1_pos': 0.563, 'f1_neu': 0.521, 'f1_neg': 0.599, 'eval_score': 0.563}
{'n_estimators': 72, 'max_depth': 50, 'min_samples_split': 9, 'min_samples_leaf': 9}
{'accuracy': 0.555, 'f1_pos': 0.555, 'f1_neu': 0.514, 'f1_neg': 0.595, 'eval_score': 0.555}
{'n_estimators': 59, 'max_depth': 97, 'min_samples_split': 3, 'min_samples_leaf': 8}
{'accuracy': 0.556, 'f1_pos': 0.562, 'f1_neu': 0.513, 'f1_neg': 0.592, 'eval_score': 0.561}
{'n_estimators': 66, 'max_depth': 38, 'min_samples_split': 3, 'min_samples_leaf': 4}
{'accuracy': 0.56, 'f1_pos': 0.561, 'f1_neu': 0.517, 'f1_neg': 0.599, 'eval_score': 0.561}
{'n_estimators': 63, 'max_depth': 72, 'min_samples_split': 5, 'min_samples_leaf': 3}
{'accuracy': 0.562, 'f1_pos': 0.576, 'f1_neu': 0.511, 'f1_neg': 0.596, 'eval_score': 0.573}
{'n_estimators': 89, 'max_depth': 91, 'min_samples_split': 4, 'min_samples_leaf': 8}
{'accuracy': 0.556, 'f1_pos': 0

In [ ]:
# With PCA,  Bayesian Optimization, and Random Forest

def objective_func_RF_PCA(args):
    n_estimators = args['n_estimators']
    max_depth = args['max_depth']
    min_samples_split = args['min_samples_split']
    min_samples_leaf = args['min_samples_leaf']
    n = args['n']

    par_dict = { 'n': n, 'n_estimators': n_estimators, 'max_depth': max_depth, 'min_samples_split': min_samples_split, 'min_samples_leaf': min_samples_leaf}
    print(par_dict)
    
    clf = RandomForestClassifier(n_estimators = n_estimators, max_depth = max_depth, min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split, n_jobs = -1)
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid_PCA(train1_pr_df['Doc Text'],train1_pr_df['Sentiment'], clf, n) 
    eval_score = round(mean([0.4*accuracy, 1.6*f1_Positive]),3)
    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}

    print(eval_dict)
    print('\n')    
    return -(eval_score)

space = {'n_estimators': hp.choice('n_estimators', np.arange(20,101, step =1)), 
         'max_depth': hp.choice('max_depth', np.arange(20,101, step =1)), 
#         'criterion': hp.choice('criterion',['gini', 'entropy']),
#         'max_features': hp.choice('max_features', ['auto', 'sqrt']),
        'min_samples_split': hp.choice('min_samples_split', np.arange(2,12, step =1)),
         'min_samples_leaf' : hp.choice('min_samples_leaf',np.arange(1,12, step =1)),
        'n': hp.choice('n', np.arange(3,203, step =20))
        }
                                
                                
                                
best_classifier_RF_PCA = fmin(objective_func_RF_PCA, space, algo=tpe.suggest, max_evals=30)
print(best_classifier_RF_PCA)

{'n': 63, 'n_estimators': 99, 'max_depth': 63, 'min_samples_split': 7, 'min_samples_leaf': 4}
{'accuracy': 0.53, 'f1_pos': 0.548, 'f1_neu': 0.481, 'f1_neg': 0.56, 'eval_score': 0.544}
{'n': 143, 'n_estimators': 43, 'max_depth': 93, 'min_samples_split': 4, 'min_samples_leaf': 3}
{'accuracy': 0.534, 'f1_pos': 0.551, 'f1_neu': 0.492, 'f1_neg': 0.56, 'eval_score': 0.548}
{'n': 63, 'n_estimators': 99, 'max_depth': 50, 'min_samples_split': 9, 'min_samples_leaf': 1}
{'accuracy': 0.542, 'f1_pos': 0.559, 'f1_neu': 0.494, 'f1_neg': 0.571, 'eval_score': 0.556}
{'n': 163, 'n_estimators': 77, 'max_depth': 98, 'min_samples_split': 10, 'min_samples_leaf': 7}
{'accuracy': 0.555, 'f1_pos': 0.553, 'f1_neu': 0.523, 'f1_neg': 0.587, 'eval_score': 0.553}
{'n': 143, 'n_estimators': 74, 'max_depth': 58, 'min_samples_split': 9, 'min_samples_leaf': 2}
{'accuracy': 0.544, 'f1_pos': 0.55, 'f1_neu': 0.508, 'f1_neg': 0.573, 'eval_score': 0.549}
{'n': 143, 'n_estimators': 62, 'max_depth': 99, 'min_samples_split': 9

In [ ]:
# # WITH LDA
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# from math import sqrt
# lda = LinearDiscriminantAnalysis(n_components = None)
# x_lda = lda.fit_transform(obama_train_pr_tfidf, obama_train_pr_df['Sentiment'])

In [ ]:
# # With LDA, Bayesian Optimization, and Random Forest

# def objective_func(args):
#     n_estimators = args['n_estimators']
#     max_depth = args['max_depth']
#     min_samples_split = args['min_samples_split']
#     min_samples_leaf = args['min_samples_leaf']
# #     pca = PCA(n_components=args['n'])
# #     x_pca1 = pca.fit_transform(obama_train_pr_tfidf)

 
#     clf = RandomForestClassifier(n_estimators = n_estimators, max_depth = max_depth, min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split, n_jobs = -1)
    
#     temp = obama_train_pr_df['Sentiment'].apply(lambda x: 1 if x == 'Positive' else (3 if x == 'Negative' else 2))
#     temp_f1 = cross_val_score(clf, x_lda, temp, cv = 5, scoring = 'f1_macro')
#     f1 = mean(temp_f1)
#     return -(f1)
# space = {'n_estimators': hp.choice('n_estimators', np.arange(20,101, step =1)), 
#          'max_depth': hp.choice('max_depth', np.arange(20,101, step =1)), 
# #         'criterion': hp.choice('criterion',['gini', 'entropy']),
# #         'max_features': hp.choice('max_features', ['auto', 'sqrt']),
#         'min_samples_split': hp.choice('min_samples_split', np.arange(2,12, step =1)),
#          'min_samples_leaf' : hp.choice('min_samples_leaf',np.arange(1,12, step =1))
# #         'n': hp.choice('n', np.arange(3,20, step =1))
#         }
                                
                                
                                
# best_classifier = fmin(objective_func, space, algo=tpe.suggest, max_evals=30)
# print(best_classifier)# With Bayesian Optimization, and Random Forest



In [ ]:
# obama_train2_pr_tfidf, obama_val_pr_tfidf, obama_train2_pr_class, obama_val_pr_class = train_test_split(obama_train_pr_tfidf, obama_train_pr_df['Sentiment'], test_size = 0.2)

In [ ]:
# x_lda2 = lda.fit_transform(obama_train2_pr_tfidf, obama_train2_pr_class)

In [ ]:
# # With LDA, Bayesian Optimization, and Random Forest

# def objective_func(args):
#     n_estimators = args['n_estimators']
#     max_depth = args['max_depth']
#     min_samples_split = args['min_samples_split']
#     min_samples_leaf = args['min_samples_leaf']
# #     pca = PCA(n_components=args['n'])
# #     x_pca1 = pca.fit_transform(obama_train_pr_tfidf)

 
#     clf = RandomForestClassifier(n_estimators = n_estimators, max_depth = max_depth, min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split, n_jobs = -1)
    
# #     temp = obama_train_pr_df['Sentiment'].apply(lambda x: 1 if x == 'Positive' else (3 if x == 'Negative' else 2))
#     temp_f1 = cross_val_score(clf, x_lda2, obama_train2_pr_class, cv = 5, scoring = 'f1_macro')
#     f1 = mean(temp_f1)
#     return -(f1)
# space = {'n_estimators': hp.choice('n_estimators', np.arange(20,101, step =1)), 
#          'max_depth': hp.choice('max_depth', np.arange(20,101, step =1)), 
# #         'criterion': hp.choice('criterion',['gini', 'entropy']),
# #         'max_features': hp.choice('max_features', ['auto', 'sqrt']),
#         'min_samples_split': hp.choice('min_samples_split', np.arange(2,12, step =1)),
#          'min_samples_leaf' : hp.choice('min_samples_leaf',np.arange(1,12, step =1))
# #         'n': hp.choice('n', np.arange(3,20, step =1))
#         }
                                
                                
                                
# best_classifier = fmin(objective_func, space, algo=tpe.suggest, max_evals=30)
# print(best_classifier)# With Bayesian Optimization, and Random Forest



In [ ]:
# x_val_lda = lda.transform(obama_val_pr_tfidf)
# x_val_lda_df = pd.DataFrame(data = x_val_lda)

In [ ]:
# c = lambda b : 1 if b == 0 else b
# d = lambda b : 2 if b <= 1 else b
# bc = RandomForestClassifier(max_depth = c(best_classifier['max_depth']), min_samples_leaf = c(best_classifier['min_samples_leaf']), min_samples_split = d(best_classifier['min_samples_split'])
#                             ,n_estimators = best_classifier['n_estimators'])

# bc.fit(x_lda2, obama_train2_pr_class)

# y_pred = bc.predict(x_val_lda)

# accuracy_score(obama_val_pr_class, y_pred)

In [ ]:
# lda_clf = lda.fit(obama_train2_pr_tfidf, obama_train2_pr_class)
# y_pred_lda_clf = lda_clf.predict(obama_val_pr_tfidf)
# # y_pred_lda_clf

# accuracy_score(obama_val_pr_class, y_pred_lda_clf)

### Light Gradient Boosting

In [ ]:
import lightgbm as lgbm
from lightgbm.sklearn import LGBMClassifier

In [ ]:
# With Bayesian Optimization, and Light GB

def objective_func_LGB(args):
    boosting_type= args['boosting_type']
    n_estimators = args['n_estimators']
    max_depth = args['max_depth']
    learning_rate = args['learning_rate']
    subsample = args['subsample']
    min_split_gain = args['min_split_gain']

    par_dict = {'boosting_type': boosting_type, 'n_estimators': n_estimators, 'max_depth': max_depth, 'learning_rate': learning_rate, 'subsample': subsample, 'min_split_gain': min_split_gain}
    print(par_dict)
    
    clf = LGBMClassifier(boosting_type = boosting_type, class_weight  = 'balanced', num_leaves = 2**max_depth, n_estimators = n_estimators, max_depth = max_depth, subsample=subsample, learning_rate=learning_rate, min_split_gain = min_split_gain)
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid(train1_pr_df['Doc Text'],train1_pr_df['Sentiment'], clf) 
    eval_score = round(mean([0.4*accuracy, 1.6*f1_Positive]),3)
    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}

    print(eval_dict)
    print('\n')    
    return -(eval_score)

space = {
    'boosting_type' : hp.choice('boosting_type', ['gbdt', 'dart', 'goss']),
    'n_estimators': hp.choice('n_estimators', np.arange(50,501, step =1)),
    'learning_rate':  hp.uniform('learning_rate', 0,0.2), 
    'max_depth': hp.choice('max_depth', np.arange(3, 10, step =1)),   
     'subsample': hp.choice('subsample', np.linspace(0.8, 1, num=4)),
    'min_split_gain': hp.choice('min_split_gain' , np.arange(0,6, step =1))}
                                
                                
                                
best_classifier_LGB = fmin(objective_func_LGB, space, algo=tpe.suggest, max_evals=25)
print(best_classifier_LGB)

{'boosting_type': 'gbdt', 'n_estimators': 71, 'max_depth': 3, 'learning_rate': 0.14471246871729218, 'subsample': 0.9333333333333333, 'min_split_gain': 1}
{'accuracy': 0.528, 'f1_pos': 0.554, 'f1_neu': 0.464, 'f1_neg': 0.561, 'eval_score': 0.549}
{'boosting_type': 'goss', 'n_estimators': 362, 'max_depth': 4, 'learning_rate': 0.16377313302510013, 'subsample': 0.8666666666666667, 'min_split_gain': 5}
{'accuracy': 0.501, 'f1_pos': 0.541, 'f1_neu': 0.457, 'f1_neg': 0.503, 'eval_score': 0.533}
{'boosting_type': 'gbdt', 'n_estimators': 82, 'max_depth': 8, 'learning_rate': 0.06817247993416801, 'subsample': 0.8, 'min_split_gain': 1}
{'accuracy': 0.531, 'f1_pos': 0.562, 'f1_neu': 0.47, 'f1_neg': 0.556, 'eval_score': 0.556}
{'boosting_type': 'dart', 'n_estimators': 284, 'max_depth': 4, 'learning_rate': 0.09073003384180156, 'subsample': 0.8, 'min_split_gain': 2}
{'accuracy': 0.53, 'f1_pos': 0.559, 'f1_neu': 0.468, 'f1_neg': 0.558, 'eval_score': 0.553}
{'boosting_type': 'dart', 'n_estimators': 442,

In [ ]:
# With PCA, Bayesian Optimization, and Light GB

def objective_func_LGB_PCA(args):
    boosting_type= args['boosting_type']
    n_estimators = args['n_estimators']
    max_depth = args['max_depth']
    learning_rate = args['learning_rate']
#     subsample = args['subsample']
    min_split_gain = args['min_split_gain']
    n = args['n']
    
    par_dict = {'n': n, 'n_estimators': n_estimators, 'max_depth': max_depth, 'learning_rate': learning_rate, 'subsample': 1, 'min_split_gain': min_split_gain} 
    print(par_dict)
    
    clf = LGBMClassifier(boosting_type = boosting_type, class_weight  = 'balanced', num_leaves = 2**max_depth, n_estimators = n_estimators, max_depth = max_depth, subsample=1, learning_rate=learning_rate, min_split_gain = min_split_gain)
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid_PCA(train1_pr_df['Doc Text'],train1_pr_df['Sentiment'], clf, n) 
    eval_score = round(mean([0.4*accuracy, 1.6*f1_Positive]),3)
    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}

    print(eval_dict)
    print('\n')    
    return -(eval_score)

space = {
    'boosting_type' : hp.choice('boosting_type', ['gbdt', 'dart', 'goss']),
    'n_estimators': hp.choice('n_estimators', np.arange(50,501, step =1)),
    'learning_rate':  hp.uniform('learning_rate', 0,0.2), 
    'max_depth': hp.choice('max_depth', np.arange(3, 10, step =1)),   
#      'subsample': hp.choice('subsample', np.linspace(0.8, 1, num=4)),
    'min_split_gain': hp.choice('min_split_gain' , np.arange(0,6, step =1)),
        'n': hp.choice('n', np.arange(3,203, step =20))}
                                
                                
                                
best_classifier_LGB_PCA = fmin(objective_func_LGB_PCA, space, algo=tpe.suggest, max_evals=35)
print(best_classifier_LGB_PCA)

{'n': 43, 'n_estimators': 154, 'max_depth': 9, 'learning_rate': 0.07686217650097071, 'subsample': 1, 'min_split_gain': 4}
{'accuracy': 0.53, 'f1_pos': 0.56, 'f1_neu': 0.482, 'f1_neg': 0.545, 'eval_score': 0.554}
{'n': 123, 'n_estimators': 345, 'max_depth': 4, 'learning_rate': 0.19585309229143938, 'subsample': 1, 'min_split_gain': 0}
{'accuracy': 0.549, 'f1_pos': 0.555, 'f1_neu': 0.509, 'f1_neg': 0.582, 'eval_score': 0.554}
{'n': 63, 'n_estimators': 73, 'max_depth': 4, 'learning_rate': 0.07595858330008012, 'subsample': 1, 'min_split_gain': 2}
{'accuracy': 0.534, 'f1_pos': 0.552, 'f1_neu': 0.5, 'f1_neg': 0.552, 'eval_score': 0.548}
{'n': 63, 'n_estimators': 425, 'max_depth': 8, 'learning_rate': 0.1033309421637879, 'subsample': 1, 'min_split_gain': 4}
{'accuracy': 0.522, 'f1_pos': 0.544, 'f1_neu': 0.481, 'f1_neg': 0.541, 'eval_score': 0.54}
{'n': 3, 'n_estimators': 441, 'max_depth': 6, 'learning_rate': 0.1716273624175174, 'subsample': 1, 'min_split_gain': 5}
{'accuracy': 0.463, 'f1_pos': 

In [ ]:
print(asdffs)

NameError: ignored

### Gradient Boosting

In [ ]:
# With Bayesian Optimization, and GB

def objective_func_GB(args):
    n_estimators = args['n_estimators']
    max_depth = args['max_depth']
    min_samples_split = args['min_samples_split']
    min_samples_leaf = args['min_samples_leaf']

    par_dict = { 'n_estimators': n_estimators, 'max_depth': max_depth, 'min_samples_split': min_samples_split, 'min_samples_leaf': min_samples_leaf}
    print(par_dict)
    
    clf = GradientBoostingClassifier(n_estimators = n_estimators, max_depth = max_depth, min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split)
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid(obama_train_pr_tfidf,obama_train_pr_df['Sentiment'], clf) 
    eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)
    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}

    print(eval_dict)
    print('\n')    
    return -(eval_score)

space = {'n_estimators': hp.choice('n_estimators', np.arange(20,101, step =1)), 
         'max_depth': hp.choice('max_depth', np.arange(20,101, step =1)), 
#         'criterion': hp.choice('criterion',['gini', 'entropy']),
#         'max_features': hp.choice('max_features', ['auto', 'sqrt']),
        'min_samples_split': hp.choice('min_samples_split', np.arange(2,12, step =1)),
         'min_samples_leaf' : hp.choice('min_samples_leaf',np.arange(1,12, step =1))
        }
                                
                                
                                
best_classifier_GB= fmin(objective_func_GB, space, algo=tpe.suggest, max_evals=30)
print(best_classifier_GB)

In [ ]:
# With PCA, Bayesian Optimization, and GB

def objective_func_GB(args):
    n_estimators = args['n_estimators']
    max_depth = args['max_depth']
    min_samples_split = args['min_samples_split']
    min_samples_leaf = args['min_samples_leaf']
    n = args['n']

    par_dict = { 'n': n, 'n_estimators': n_estimators, 'max_depth': max_depth, 'min_samples_split': min_samples_split, 'min_samples_leaf': min_samples_leaf}
    print(par_dict)
    
    clf = RandomForestClassifier(n_estimators = n_estimators, max_depth = max_depth, min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split, n_jobs = -1)
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid_PCA(obama_train_pr_tfidf,obama_train_pr_df['Sentiment'], clf, n) 
    eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)
    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}

    print(eval_dict)
    print('\n')    
    return -(eval_score)

space = {'n_estimators': hp.choice('n_estimators', np.arange(20,101, step =1)), 
         'max_depth': hp.choice('max_depth', np.arange(20,101, step =1)), 
#         'criterion': hp.choice('criterion',['gini', 'entropy']),
#         'max_features': hp.choice('max_features', ['auto', 'sqrt']),
        'min_samples_split': hp.choice('min_samples_split', np.arange(2,12, step =1)),
         'min_samples_leaf' : hp.choice('min_samples_leaf',np.arange(1,12, step =1)),
        'n': hp.choice('n', np.arange(3,203, step =20))
        }
                                
                                
                                
best_classifier_RF_PCA = fmin(objective_func_RF_PCA, space, algo=tpe.suggest, max_evals=30)
print(best_classifier_RF_PCA)

In [ ]:
from imblearn.over_sampling import SMOTE


sm = SMOTE(sampling_strategy='not majority',random_state=42)
romney_train_pr_tfidf_smote, romney_train_pr_class = sm.fit_sample(romney_train_pr_tfidf, romney_train_pr_df['Sentiment'] )


sm = SMOTE(sampling_strategy='not majority',random_state=42)
obama_train_pr_tfidf_smote, obama_train_pr_class = sm.fit_sample(obama_train_pr_tfidf, obama_train_pr_df['Sentiment'])